In [9]:
import hashlib
import sys

def sha256(data: bytes) -> bytes:
    return hashlib.sha256(data).digest()

def leaf_hash(value: str) -> bytes:
    # Prefix leaves to separate from internal node hashes
    return sha256(b'\x00' + value.encode('utf-8'))

def node_hash(left: bytes, right: bytes) -> bytes:
    # Prefix internal nodes
    return sha256(b'\x01' + left + right)

class MerkleTree:
    def __init__(self, values):
        # values: list of strings
        self.leaves = [leaf_hash(v) for v in values]
        if not self.leaves:
            self.layers = [[sha256(b'')]]  # root for empty tree (arbitrary)
        else:
            self._build_layers()

    def _build_layers(self):
        layers = [self.leaves]
        while len(layers[-1]) > 1:
            cur = layers[-1]
            nxt = []
            i = 0
            while i < len(cur):
                left = cur[i]
                if i + 1 < len(cur):
                    right = cur[i+1]
                else:
                    # duplicate last if odd count
                    right = left
                nxt.append(node_hash(left, right))
                i += 2
            layers.append(nxt)
        self.layers = layers

    def root(self) -> bytes:
        return self.layers[-1][0] if self.layers else sha256(b'')

    def get_proof_by_index(self, index: int):
        """
        index: 0-based leaf index
        returns: list of tuples (direction, sibling_hash_bytes)
                 direction is 'L' if sibling is left of current node,
                              'R' if sibling is right of current node.
        """
        if index < 0 or index >= len(self.leaves):
            raise IndexError("leaf index out of range")

        proof = []
        idx = index
        for level in range(len(self.layers)-1):  # for each level where pairs exist
            layer = self.layers[level]
            pair_idx = idx ^ 1  # sibling index (if idx even, sibling idx+1; if odd, sibling idx-1)
            # If sibling out of range, it means it was duplicated, so sibling == node itself
            if pair_idx >= len(layer):
                sibling_hash = layer[idx]
            else:
                sibling_hash = layer[pair_idx]
            # determine direction: if sibling index < idx then sibling is left of current
            direction = 'L' if pair_idx < idx else 'R'
            proof.append((direction, sibling_hash))
            # move up
            idx //= 2
        return proof

    def get_proof_by_value(self, value: str):
        # find first occurrence
        target = leaf_hash(value)
        for i, leaf in enumerate(self.leaves):
            if leaf == target:
                return i, self.get_proof_by_index(i)
        return None, None

def proof_to_hex(proof):
    # convert proof entries to human-readable hex
    return [(d, h.hex()) for (d, h) in proof]

def verify_proof(value: str, proof, root_hex: str) -> bool:
    cur = leaf_hash(value)
    for direction, sibling_hex in proof:
        sibling = bytes.fromhex(sibling_hex)
        if direction == 'L':
            cur = node_hash(sibling, cur)
        else:
            cur = node_hash(cur, sibling)
    return cur.hex() == root_hex

# --- Flexible command-line / CF-style input handling ---
def main():
    data = sys.stdin.read().strip().split()
    if not data:
        print("No input provided. See example in source code.")
        return

    it = iter(data)
    try:
        n = int(next(it))
    except StopIteration:
        print("Bad input.")
        return

    values = []
    for _ in range(n):
        try:
            values.append(next(it))
        except StopIteration:
            values.append("")  # if missing tokens treat as empty string

    tree = MerkleTree(values)
    root_hex = tree.root().hex()

    # read q
    try:
        q = int(next(it))
    except StopIteration:
        q = 0

    outputs = []
    for _ in range(q):
        try:
            token = next(it)
        except StopIteration:
            break
        # try interpret as integer index (1-based)
        is_index = False
        try:
            idx = int(token)
            is_index = True
        except ValueError:
            is_index = False

        if is_index:
            idx0 = idx - 1
            if idx0 < 0 or idx0 >= len(values):
                outputs.append("NOTFOUND")
            else:
                proof = tree.get_proof_by_index(idx0)
                outputs.append(f"ROOT {root_hex}")
                outputs.append(f"PROOF {len(proof)}")
                for d, h in proof_to_hex(proof):
                    outputs.append(f"{d} {h}")
        else:
            pos, proof = tree.get_proof_by_value(token)
            if proof is None:
                outputs.append("NOTFOUND")
            else:
                outputs.append(f"ROOT {root_hex}")
                outputs.append(f"PROOF {len(proof)}")
                for d, h in proof_to_hex(proof):
                    outputs.append(f"{d} {h}")

    sys.stdout.write("\n".join(outputs))

if __name__ == "__main__":
    main()


No input provided. See example in source code.
